In [1]:
try:
    ! pip install -r requirements.txt
except:
    pass

In [2]:
import pandas as pd
import numpy as np
import string
from collections import Counter
import time
from IPython.display import clear_output
import warnings; warnings.simplefilter('ignore')
from scipy.optimize import linprog

In [3]:
def initalizing (A,b,ConsType):
    n_const,n_var = 0,0
    n_const,n_var = A.shape #The parameters
    #getting the indices of artificial, basis and slack variables
    slack_var = []
    Iden_loc = [0]*n_const
    artificial_var = []
    # If there are negative b values then change the sign of the values in corresponding row
    for i in range(n_const):
        if (b[i]<0):
            A[i,:] = -1*A[i,:]
            b[i] = -1*b[i]
            ConsType[i] = -1*ConsType[i]
    for i in range(n_const):
        if (ConsType[i]>0):
            n_var = n_var + 1
            dum = np.zeros((n_const,1))
            A = np.append(A,dum,axis=1)
            slack_var.append(n_var-1)
            Iden_loc[i]=(n_var-1)
            A[i,n_var-1] = 1
        elif (ConsType[i]<0):
            n_var = n_var + 1
            dum = np.zeros((n_const,1))
            A = np.append(A,dum,axis=1)
            slack_var.append(n_var-1)
            A[i,n_var-1] = -1
    # Adding the artificial and Identity variables into the list
    for i in range(n_const):
        if (ConsType[i]<0 or Iden_loc[i]==0):
            n_var = n_var + 1
            dum = np.zeros((n_const,1))
            A = np.append(A,dum,axis=1)
            artificial_var.append(n_var-1)
            Iden_loc[i]=(n_var-1)
            A[i,n_var-1] = 1
    print('the slack variables are:',slack_var)
    print('the artificial variables are:',artificial_var)
    print('the initial basis variables are:',Iden_loc)
    basis_var = Iden_loc
    non_basis_var = [x for x in range(n_var) if x not in basis_var] #Getting the non basis variable
    print('The non basis variable are:', non_basis_var)
    return(n_const,n_var,slack_var,Iden_loc,artificial_var,A,basis_var,non_basis_var)

In [4]:
def unboundness(rt): #Checks the unboundness
    proceed = False
    if len(rt) == 0:
        print('The problem is infeasible')
        proceed = True
    return proceed

In [5]:
def cycling(rt,vtlb): #Resolves cycling using Bland's rule
    dum = np.extract(rt == np.inf, rt)
    dum = rt[rt == np.min(dum)].index
    dum1 = [x for x in dum if x not in vtlb]
    return min(dum1)

In [6]:
def check_cycling(rt, dum, vtlb): # Checks for cycling
    count = 0
    for i in dum:
        if (i == np.inf):
            count += 1
    if count == len(dum): #if cycling exists
        return cycling(rt,vtlb)
    else:
        dum = rt[rt == np.min(dum)].index #Resolves the conflict by choosing the varaible with minimum index
        dum1 = [x for x in dum if x not in vtlb]
        return min(dum1)

In [7]:
def row_operations(tab, ib, inb, v2b, v2nb, basis,vtlb): #Row operations on the tableau
    dum = tab.loc[v2nb,v2nb]/tab.loc[v2nb,v2b] #pivoting the variable to basis 
    tab.loc[v2nb,:] = dum * tab.loc[v2nb,:]
    for i in ib + ['z']: # row operations for the pivoted row
        if i != v2nb:
            dum = -tab.loc[i,v2b]/tab.loc[v2nb,v2b]
            tab.loc[i,:] += (dum * tab.loc[v2nb,:])
    dum = basis.index(v2nb) #updating the basis,non basis indices
    basis[dum] = v2b
    inb.remove(v2b)
    inb.append(v2nb)
    vtlb.append(v2nb)
    tab.index = ['z'] + basis
    ib = basis
    return tab, ib, inb,vtlb

In [8]:
def simplex(tab, ib, inb,vtlb):
    basis = ib
    ib = sorted(ib)
    inb = sorted(inb)
    find_pivot = tab.loc['z',inb] #Find the variable to enter the basis
    dum = np.extract(find_pivot > 1e-3 , find_pivot)
    v2bi = find_pivot[find_pivot == np.max(dum)].index
    v2b = min(v2bi)
    inter = []
    for i in ib: #Find the variable to leave the basis
        if tab.loc[i,v2b]>1e-3:
            inter.append(i)
    rt = tab.loc[inter, 'B'] / tab.loc[inter, v2b]
    if unboundness(rt):
        return tab, ib, inb, True
    else:
        dum = np.extract(rt >= 0, rt)
        v2nb = check_cycling(rt, dum,vtlb)
        tab, ib, inb,vtlb = row_operations(tab, ib, inb, v2b, v2nb,basis,vtlb)
        print("Variable to basis:", v2b)
        print("Variable to non basis:",v2nb)
        return tab, ib, inb,vtlb

In [9]:
# Special case: when artificial variables are present in basis after reaching optimal value in phase 1
def artificial_pivot(tab,ib,inb,vtlb): 
    basis = ib
    ib = sorted(ib)
    inb = sorted(inb)
    for i in inb: #choosing the first non zero variable to enter the basis
        if tab.loc['z',i]!=0:
            v2b = i
            break
    rt = tab.loc[ib,'B']/tab.loc[ib,v2b]
    if unboundness(rt):
        return tab, ib, inb
    else:
        dum = np.extract(rt >= 0, rt)
        v2nb = check_cycling(rt, dum,vtlb) #non basis variable exits the basis
        tab, ib, inb,vtlb = row_operations(tab, ib, inb, v2b, v2nb,basis,vtlb)
        print("Variable to basis:", v2b)
        print("Variable to non basis:",v2nb)
        return tab, ib, inb,vtlb

In [10]:
def phase1(tableau,basis_var,non_basis_var,artificial_var,C,slack_var): #the phase 1 LP Simplex
    #Initializing the tableau for phase 1
    tableau.loc['z', basis_var] = 0
    for i in non_basis_var+['B']:
        tableau.loc['z', i] = sum(tableau.loc[artificial_var, i])
    print('Tableau for phase 1 \n',tableau)
    # Condition check to perform the pivoting operations 
    condition1 = False
    for i in non_basis_var:
        if tableau.loc['z',i]>0:
            condition1 = True
            break
    vtlb1 = [] # This variable keeps the record of the variable that entered the basis -> left the basis -> re-enters the basis
    print('The Pivoting operations done to achieve optimal value is: \n')
    while (condition1):
        tableau, basis_var, non_basis_var,vtlb1 = simplex(tableau,basis_var,non_basis_var,vtlb1)
        for i in non_basis_var:
            if tableau.loc['z',i]>1e-3:
                condition1 = True
                break
            else:
                condition1 = False
#         tableau = tableau.round(2)
    print('\n')
    if tableau.loc['z','B']<1e-2: #checking the optimality of phase 1 
        print('optimality is achieved proceed to phase 2 \n')
        # If there any artificial variables that are present in the basis
        print('Pivoting operations done to remove the artificial variables \n')
        while (len([x for x in artificial_var if x in basis_var ])>0):
            a=[x for x in artificial_var if x in basis_var ]
            b = [x for x in (tableau.columns[:-1]) if x not in artificial_var]
            print(tableau)
            ''' 
            This is where we'll deal with redundancy in the problem. After the phase 1 optimality is achieved and if 
            artificial variables are present in the basis with corresponding artificial RHS = 0 and that the only artificial
            variable present in the basis, that row can be deleted.
            '''
            if (tableau.loc[basis_var,'B'].all()>=1e-2) and (tableau.loc[a,b].all()>=1e-2).all(): 
                for i in basis_var:
                    if tableau.loc[i,'B']==0:
                        tableau = tableau.drop([i])
                        basis_var.remove(i)
                        print('x%d is eliminated and this is where the redudancy in the problem is dealt \n'%(i))
            else:
                tableau, basis_var, non_basis_var,vtlb1 = artificial_pivot(tableau,basis_var,non_basis_var,vtlb1)
                #cycling to move the artificial variables out of basis
        print('\n')
        tableau = tableau.round(2)
        print('Tableau after phase 1: \n',tableau)
        if type(C)!=str:
            # Converting the tableau from phase 1 -> phase 2
            tableau = tableau.drop(columns=artificial_var)
            dum = np.zeros((1,len(slack_var)+1))
            dum = np.append(C,dum,axis=1)
            tableau.loc['z',:] = dum
            for i in basis_var:
                if tableau.loc['z',i] != 0:
                    dum = tableau.loc['z',i]
                    tableau.loc['z',:] -= dum*tableau.loc[i,:]
            tableau.loc['z',:] = -1*tableau.loc['z',:]
            print('the tableau for phase 2 \n',tableau)
        return tableau,basis_var,non_basis_var,vtlb1
    else:
        print('The LP is infeasible \n')
        return tableau,basis_var,non_basis_var,True

In [11]:
def phase2(tableau,basis_var,non_basis_var,artificial_var=[]): #the phase 2 LP Simplex
    #condition to check for positive reduced cost
    non_basis_var = [x for x in non_basis_var if x not in artificial_var]
    condition = False
    for i in non_basis_var:
        if tableau.loc['z',i]>0:
            condition = True
            break
    vtlb2 = []
    print('The Pivoting operations done to achieve optimal value is: \n')
    while (condition):
        tableau, basis_var, non_basis_var,vtlb2 = simplex(tableau,basis_var,non_basis_var,vtlb2)
        for i in non_basis_var:
            if tableau.loc['z',i]>1e-3:
                condition = True
                break
            else:
                condition = False
        if vtlb2==True:
            break
    return tableau, basis_var, non_basis_var,vtlb2

In [12]:
def check_efficiency (tableau,non_basis_var):
    '''
    This is used to check the efficient non basic variable that could be used for the pivoting operations in Phase 3
    max e^(T)v s.t Rz+R_{j}d+Iv = 0
    '''
    enbv = []
    for finder in non_basis_var:
        cost_indices = list(string.ascii_lowercase[:C.shape[0]])
        dum = tableau.loc[cost_indices,non_basis_var]
        dummer = -tableau1.loc[cost_indices,finder]
        dum['chosen_index'] = dummer
        vs = []
        for i in range(number_w):
            vs.append("V"+str(i))
        dummer = pd.DataFrame(W0,index=cost_indices,columns=vs)
        dum = dum.join(dummer)
        dumb = np.zeros(len(cost_indices))
        dumc = np.ones(len(cost_indices))
        Cost_function_phase_22 = np.append(np.zeros(len(non_basis_var)+1),np.ones(len(vs)))
#         clear_output(wait=True)
#         phase31A = dum
#         phase31b = dumb
#         phase31c = np.concatenate((np.zeros(len(non_basis_var)+1),dumc),axis=0)
#         phase31b = np.asmatrix(phase31b)
#         phase31c = -np.asmatrix(phase31c)
#         res = linprog(phase31c,phase31A,phase31b,method = 'simplex')
# #         print(res)
#         if res.fun==0 and res.status==0:
#             enbv.append(finder)
        n_const4,n_var4,slack_var4,Iden_loc4,artificial_var4,A4,basis_var4,non_basis_var4 = initalizing(dum,dumb,dumc)
        clear_output(wait=True)
        tableau4 = pd.DataFrame(A4)
        Cost_function_phase_22 = pd.DataFrame(Cost_function_phase_22)
        tableau4 = pd.concat([Cost_function_phase_22.T, tableau4])
        tableau4 = tableau4.fillna(0)
        tableau4.index = list(range(n_const4+1))
        dum = np.append(np.zeros((1, 1)), np.asmatrix(dumb).T, axis  = 0)
        dum = pd.DataFrame(dum)
        tableau4['B'] = dum
        tableau4.index = ['z'] + basis_var4
        print('Proceed to phase two')
        tableau4.loc['z',:] = tableau4.loc['z',:]
        tableau4,basis_var4,non_basis_var4,checker=phase2(tableau4,basis_var4,non_basis_var4,artificial_var4)
        clear_output(wait=True)
        if checker!=True and tableau4.loc['z','B']==0:
            enbv.append(finder)
    return(enbv)

In [13]:
def feasible_pivot (tab,basis_var,non_basis_var,e_non_basis_var):
    '''
    Performs the pivoting operations for the obtained efficient non basic variable.
    '''
    updates_b = []
    updates_inb = []
    updates_tab = []
    for v2b in e_non_basis_var:
        ib = basis_var.copy()
        basis = basis_var.copy()
        inb = non_basis_var.copy()
        tab1 = tab.copy()
        ib = sorted(ib)
        inter = []
        try:
            for i in ib: #Find the variable to leave the basis
                if tab1.loc[i,v2b]>1e-3:
                    inter.append(i)
            rt = tab1.loc[inter, 'B'] / tab1.loc[inter, v2b]
            dum = np.extract(rt >= 0, rt)
            dum = rt[rt == np.min(dum)].index
            v2nb = min(dum)
            dum = tab1.loc[v2nb,v2nb]/tab1.loc[v2nb,v2b]
            tab1.loc[v2nb,:] = dum * tab1.loc[v2nb,:]
            for i in ib + ['z'] + list(string.ascii_lowercase[:C.shape[0]]):
                if i != v2nb:
                    dum = -tab1.loc[i,v2b]/tab1.loc[v2nb,v2b]
                    tab1.loc[i,:] += (dum * tab1.loc[v2nb,:])
            dum = basis.index(v2nb) #updating the basis,non basis indices
            basis[dum] = v2b
            inb.remove(v2b)
            inb.append(v2nb)
            tab1.index = list(string.ascii_lowercase[:C.shape[0]])+['z']+basis
            updates_b.append(basis)
            updates_inb.append(inb)
            updates_tab.append(tab1)
        except:
            pass
    return(updates_b,updates_inb,updates_tab)

In [14]:
tic = time.time()

In [15]:
#Input the problem in the minimization format min Cx s.t. Ax<=b
A = np.array([[1,1,0],
              [0,1,0],
              [1,-1,1]])
b = np.array([[1],[2],[4]])
C = np.array([[-1,-2,0],[-1,0,2],[1,0,-1]])
ConsType = np.array([1,1,1])
'''
For constraint type:
 if <= or < write 1
 if >= or > write -1
 if = write 0
'''


n_const1,n_var1,slack_var1,Iden_loc1,artificial_var1,A1,basis_var1,non_basis_var1 = initalizing(A,b,ConsType)

the slack variables are: [3, 4, 5]
the artificial variables are: []
the initial basis variables are: [3, 4, 5]
The non basis variable are: [0, 1, 2]


In [16]:
#converting the obatined the matrices into a tableau format
tableau1 = pd.DataFrame(A1)
dum = np.zeros((1, n_var1 - C.shape[1]))
dums = []
for i in range(C.shape[0]):
    dumse = np.concatenate((C[i],dum[0]),axis=0)
    dums.append(dumse)
Final_cost = dums
Final_cost = np.array(Final_cost)
dums.append(np.zeros(len(dums[1])))
dums = np.array(dums)
dum = pd.DataFrame(dums)
tableau1 = pd.concat([dum, tableau1])
tableau1.index = list(string.ascii_lowercase[:C.shape[0]])+['z']+basis_var1
dums = np.zeros((1,dum.shape[0]))
dums = np.concatenate((dums,b.T),axis=1)
dums = pd.DataFrame(dums)
dums.columns = list(string.ascii_lowercase[:C.shape[0]])+['z']+basis_var1
tableau1['B'] = dums.T
print('The converted matrices into Tableau format is: \n',tableau1)

The converted matrices into Tableau format is: 
      0    1    2    3    4    5    B
a -1.0 -2.0  0.0  0.0  0.0  0.0  0.0
b -1.0  0.0  2.0  0.0  0.0  0.0  0.0
c  1.0  0.0 -1.0  0.0  0.0  0.0  0.0
z  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3  1.0  1.0  0.0  1.0  0.0  0.0  1.0
4  0.0  1.0  0.0  0.0  1.0  0.0  2.0
5  1.0 -1.0  1.0  0.0  0.0  1.0  4.0


In [17]:
MOLPphase1, MOLPphase2 = True,True

# Phase 1

In [18]:
if MOLPphase1 and MOLPphase2:
    checker = False
    tableau1_d,basis_var_d= tableau1.loc['z':,],basis_var1.copy()
    non_basis_var_d,artificial_var_d = non_basis_var1.copy(),artificial_var1.copy()
    if (len(artificial_var1)!=0):
        print('start phase 1')
        tableau1_d,basis_var_d,non_basis_var_d,checker=phase1(tableau1_d,basis_var_d,
                                                              non_basis_var_d,artificial_var_d,'Nope',None)
    if checker:
        MOLPphase1 = False          

# Phase 2


In [19]:
'''
In this we are formulating the following LP problem
min u^(T)b + W^(T)Cx0 s.t. u^(T)A + W^(T)C >=0, w>=e
where X0 is the basis obtained from phase 1
'''
if MOLPphase1 and MOLPphase2:
    X0 = []
    for i in tableau1_d.columns:
        if i in basis_var_d:
            X0.append(tableau1_d.loc[i]['B'])
        elif i in non_basis_var_d:
            X0.append(0)
    X0 = np.array(X0)
    dum = np.matmul(Final_cost,X0)
    number_w = dum.shape[0]
    dum = np.asmatrix(dum)
    Cost_function_phase_2 = np.concatenate((b.T,dum),axis=1)
    A_phase_2 = []
    Constraints_phase_2 = []
    for i in range(A1.T.shape[0]-len(slack_var1)-len(artificial_var1)):
        dumer = np.concatenate((A1.T[i],Final_cost.T[i]),axis=0)
        A_phase_2.append(dumer)
        Constraints_phase_2.append(0)
    dum = A_phase_2[0].shape[0]
    W0 = np.eye(number_w)
    for i in range(number_w):
        dumer = np.concatenate((np.zeros(dum - number_w),W0[i]),axis=0)
        A_phase_2.append(dumer)
        Constraints_phase_2.append(1)
    A_phase_2 = np.array(A_phase_2)
    place_W = list(range(number_w,A_phase_2.shape[0]))
    Constraints_phase_2 = np.asmatrix(Constraints_phase_2)
    Constraints_phase_2 = Constraints_phase_2.T
    Constraints_type_phase_2  = -1*np.ones(A_phase_2.shape[0])
    phase21A = -1*A_phase_2[0:number_w+1]
    phase21B = Constraints_phase_2[0:number_w+1]
    boundsw = (1,None)
    boundsu = (0,None)
    try:
        res = linprog(Cost_function_phase_2,phase21A,phase21B,
                  bounds = (boundsu,boundsu,boundsu,boundsu,boundsu,boundsu,boundsu,boundsu,boundsu,
                            #boundsu,boundsu,
                            #boundsw,
                            boundsw,boundsw,boundsw),
                  method='revised simplex')
        if res.message!='Optimization terminated successfully.':
            W_star = np.ones(len(place_w))
        else:
            W_star = res.x[9:]
    except:
        W_star = np.ones(number_w)
        pass
    '''
    From the obtained W* the following creates the LP problem
    min W*^(T)Cx s.t. Ax=b,x>=0
    '''
    print('The obtained W* is:',W_star)
    Cost_function_phase_21 = np.matmul(W_star,C)
    Cost_function_phase_21 = np.asmatrix(Cost_function_phase_21)
    tableau3 = tableau1.iloc[C.shape[0]:,:]
    dum = np.zeros((1, n_var1 - Cost_function_phase_21.shape[1]))
    dum = np.append(Cost_function_phase_21, dum, axis = 1)
    dum = pd.DataFrame(dum)
    tableau3.iloc[0,:-1] = dum.iloc[0,:]
    print('The converted matrices into Tableau format is: \n',tableau3)
    checker = False
    if (len(artificial_var1)==0):
        print('Proceed to phase two')
        tableau3.loc['z',:] = -1*tableau3.loc['z',:]
        tableau3,basis_var1,non_basis_var1,_=phase2(tableau3,basis_var1,non_basis_var1,artificial_var1)
        ph2=0
    else:
        print('start phase 1')
        tableau3,basis_var1,non_basis_var1,_=phase1(tableau3,basis_var1,non_basis_var1,artificial_var1,
                                                    Cost_function_phase_21,slack_var1)
        ph2=1
    if ph2 ==1:
        tableau3.loc['z',:] = -1*tableau.loc['z',:]
        print(tableau3)
        tableau3,basis_var1,non_basis_var1,checker=phase2(tableau3,basis_var1,non_basis_var1,artificial_var1)
    if checker:
        MOLPphase2 = False
    else:
        '''
        The following finds the C_{B} and C_{N} of the cost matrix using the basis found from the above problem
        '''
        tableau1.index = list(string.ascii_lowercase[:C.shape[0]])+['z']+basis_var1
        tableau1.update(tableau3)
        tableau1 = tableau1.drop(columns=artificial_var1)
        for j in list(string.ascii_lowercase[:C.shape[0]]):
            for i in basis_var1:
                if tableau1.loc[j,i] != 0:
                    dum = tableau1.loc[j,i]
                    tableau1.loc[j,:] -= dum*tableau1.loc[i,:]
        print('The converted matrices into Tableau format is: \n',tableau1)

The obtained W* is: [1. 1. 1.]
The converted matrices into Tableau format is: 
      0    1    2    3    4    5    B
z -1.0 -2.0  1.0  0.0  0.0  0.0  0.0
3  1.0  1.0  0.0  1.0  0.0  0.0  1.0
4  0.0  1.0  0.0  0.0  1.0  0.0  2.0
5  1.0 -1.0  1.0  0.0  0.0  1.0  4.0
Proceed to phase two
The Pivoting operations done to achieve optimal value is: 

Variable to basis: 1
Variable to non basis: 3
The converted matrices into Tableau format is: 
      0    1    2    3    4    5    B
a  1.0  0.0  0.0  2.0  0.0  0.0  2.0
b -1.0  0.0  2.0  0.0  0.0  0.0  0.0
c  1.0  0.0 -1.0  0.0  0.0  0.0  0.0
z -1.0  0.0 -1.0 -2.0 -0.0 -0.0 -2.0
1  1.0  1.0  0.0  1.0  0.0  0.0  1.0
4 -1.0  0.0  0.0 -1.0  1.0  0.0  1.0
5  2.0  0.0  1.0  1.0  0.0  1.0  5.0


# Phase 3

In [20]:
if MOLPphase1 and MOLPphase2:
    '''
    L1 will the set of efficient basis for the MOLP problem
    '''
    L1 = [basis_var1]
    L1b = [non_basis_var1]
    L1tb = [tableau1] 

    all_feasible_basis = 0
    cost_indices = list(string.ascii_lowercase[:C.shape[0]])
    while(all_feasible_basis<len(L1) and all_feasible_basis<=12):
        '''
        worst case is 2^n efficient basis. the efficient basis are limited to 12. 
        '''
        L2 = L1[all_feasible_basis]
        L2nb = L1b[all_feasible_basis]
        L2tb = L1tb[all_feasible_basis]
        efficient_non_basis_var = check_efficiency(L2tb,L2nb)
        eligible_basis,eligible_non_basis = [], []
        eligible_basis,eligible_non_basis,eligible_tableau = feasible_pivot(L2tb,L2,L2nb,efficient_non_basis_var)
        for reject in range(len(eligible_basis)):
            checking = 0
            for accept in range(len(L1)):
                if Counter(eligible_basis[reject])==Counter(L1[accept]):
                    checking+=1
            if checking==0 and all_feasible_basis<=12:
                L1.append(eligible_basis[reject])
                L1b.append(eligible_non_basis[reject])
                L1tb.append(eligible_tableau[reject])
        all_feasible_basis+=1
#         clear_output(wait=True)
    print('The efficient bases are:',L1)
else:
    print('There are only weakly dohominated solutions. There is no ideal efficient or strongly efficient solutions')

The efficient bases are: [[1, 4, 5], [0, 4, 5], [1, 4, 2]]


In [21]:
toc = time.time()
print('The time taken to find the solutions are: %.1fs'%(toc-tic))

The time taken to find the solutions are: 1.4s
